In [ ]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
def scrape_all():
   # Initiate headless driver for deployment
    browser = Browser("chrome", executable_path="chromedriver", headless=True)
    news_title, news_paragraph = mars_news(browser)
    # Run all scraping functions and store results in a dictionary
    data = {
        "news_title": news_title,
        "news_paragraph": news_paragraph,
        "featured_image": featured_image(browser),
        "facts": mars_facts(),
        "last_modified": dt.datetime.now()
    }
    # Stop webdriver and return data
    browser.quit()
    return data
# Set the executable path and initialize the chrome browser in splinter
# executable_path = {'executable_path': 'chromedriver'}
# browser = Browser('chrome', **executable_path)

# ## Visit the mars nasa news site
def mars_news(browser):
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    # Optional delay for loading the page
    browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

    # ## Set up the HTML parser
    html = browser.html
    news_soup = soup(html, 'html.parser')

    # Add try/except for error handling
    try:
        slide_elem = news_soup.select_one('ul.item_list li.slide')

        # ## Scraping for content title
        # slide_elem.find("div", class_='content_title')

        # ### Get only text from content title
        # Use the parent element to find the first `a` tag and save it as `news_title`
        news_title = slide_elem.find("div", class_='content_title').get_text()
        

        # ## Scraping for the summary
        # Use the parent element to find the paragraph text
        news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
        
    except AttributeError:
        return None, None
    return news_title, news_p
# ## Featured Images
# ### Set up URL for space image
# Visit URL
def featured_image(browser):
    url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
    browser.visit(url)

    # ### Find and click the full image button
    full_image_elem = browser.find_by_id('full_image')[0]
    full_image_elem.click()

    # ### Find the more info button and click that
    browser.is_element_present_by_text('more info', wait_time=1)
    more_info_elem = browser.links.find_by_partial_text('more info')
    more_info_elem.click()

    # ### Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')

    # Add try/except for error handling
    try:
        # ### Find the relative image url
        img_url_rel = img_soup.select_one('figure.lede a img').get("src")
    except AttributeError:
        return None    

    # ### Add the base URL to code
    # Use the base URL to create an absolute URL
    img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
    return img_url

# ## Scrape Mars Data: Mars Facts
# ### Scraping whole table of Mars Facts
def mars_facts():
    # Add try/except for error handling
    try:
        df = pd.read_html('http://space-facts.com/mars/')[0]
    except BaseException:
        return None
    # Assign columns and set index of dataframe
    df.columns=['Description', 'Mars']
    df.set_index('Description', inplace=True)

    # Convert dataframe into HTML format, add bootstrap
    return df.to_html(classes="table table-striped")

if __name__ == "__main__":
    # If running as script, print scraped data
    print(scrape_all())


In [ ]:
import scraping
data = scraping.scrape_all()
print(data)

In [ ]:
from flask_pymongo import PyMongo
from flask import Flask, render_template
import scraping
app = Flask(__name__)
# Use flask_pymongo to set up mongo connection
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
mongo = PyMongo(app)
@app.route("/")
def index():
   mars = mongo.db.mars.find_one()
   return render_template("index.html", mars=mars)
@app.route("/scrape")
def scrape():
   mars = mongo.db.mars
   mars_data = scraping.scrape_all()
   mars.update({}, mars_data, upsert=True)
   return "Scraping Successful!"
if __name__ == "__main__":
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jul/2020 13:20:29] "GET / HTTP/1.1" 200 -
C:\Users\SC013\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  app.launch_new_instance()
127.0.0.1 - - [20/Jul/2020 13:20:46] "GET /scrape HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2020 13:20:50] "GET / HTTP/1.1" 200 -


In [ ]:
def featured_image(browser):
    url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
    browser.visit(url)

    # ### Find and click the full image button
    full_image_elem = browser.find_by_id('wide-image-toggle')[0]
    full_image_elem.click()

    # ### Find the more info button and click that
    #browser.is_element_present_by_text('more info', wait_time=1)
    #more_info_elem = browser.links.find_by_partial_text('more info')
    #more_info_elem.click()

    # ### Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')

    # Add try/except for error handling
    try:
        # ### Find the relative image url
        img_url_rel = img_soup.select_one('figure.lede a img').get("src")
    except AttributeError:
        return None    

    # ### Add the base URL to code
    # Use the base URL to create an absolute URL
    img_url = f'https://astrogeology.usgs.gov{img_url_rel}'
    return img_url
img_url = featured_image   
print(img_url)

In [ ]:
def featured_image(browser):
    url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
    browser.visit(url)

    # ### Find and click the full image button
    full_image_elem = browser.find_by_id('wide-image-toggle')[0]
    full_image_elem.click()

    # ### Find the more info button and click that
    #browser.is_element_present_by_text('more info', wait_time=1)
    #more_info_elem = browser.links.find_by_partial_text('more info')
    #more_info_elem.click()

    # ### Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')

    # Add try/except for error handling
    try:
        # ### Find the relative image url
        img_url_rel = img_soup.select_one('img.wide-image a img').get("src")
    except AttributeError:
        return None    

    # ### Add the base URL to code
    # Use the base URL to create an absolute URL
    img_url = f'https://astrogeology.usgs.gov{img_url_rel}'
    return img_url
    print(img_url)   
   

In [ ]:
print(img_url)   

In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
browser = Browser("chrome", executable_path="chromedriver", headless=True)
url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
browser.visit(url)

    # ### Find and click the full image button
full_image_elem = browser.find_by_id('wide-image-toggle')[0]
full_image_elem.click()
html = browser.html
img_soup = soup(html, 'html.parser')
img_url_rel = img_soup.select_one('img.wide-image').get("src")
img_url_rel    
#     img_url = f'https://astrogeology.usgs.gov{img_url_rel}'
#     return img_url
# print(featured_image)   